In [11]:
from langchain_community.tools import DuckDuckGoSearchResults
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate


import openai

In [12]:
load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_key")
openai.api_key = OPENAI_API_KEY

In [31]:
def search_web(question, num_results=4):
    search = DuckDuckGoSearchResults(output_format="list")
    results = search.invoke(question)
    return results[:num_results]

def format_document(documents):
    docs_list = []
    for doc in documents:
        try:
            docs_list.append(f"{doc['snippet']}")
        except:
            pass
    return docs_list

def rank_documents(question, documents, model_name='thenlper/gte-small'):
    model = SentenceTransformer(model_name)
    question_embedding = model.encode(question)
    doc_embeddings = model.encode(documents)

    scores = [util.cos_sim(question_embedding, doc_embedding).item() for doc_embedding in doc_embeddings]
    ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

    return ranked_docs

def compile_context(ranked_docs, top_n=2):
    context = ""
    if isinstance(ranked_docs, list):
        context = "\n".join([doc for doc, score in ranked_docs[:top_n]])
    return context

def format_prompt():
    prompt = PromptTemplate.from_template("""
        Instructions : You are an intelligent bot. Answer the question based on the provided context. If the context does not include enough information, respond with "Insufficient information.
        Question : {question}
        Context : {context}
        """)
    
    return prompt
    


In [19]:
# def RagPipeline(question):
#     if question:
#         # Searching Web
#         web_docs = search_web(question)

#         # Formating documents by extracting the snippets
#         formatted_docs = format_document(web_docs)

#         # Ranking the documents
#         rank_docs = rank_documents(question, formatted_docs)

#         # Select Top 2 documents and compile context
#         context = compile_context(rank_docs, 2)

#         # Create Prompt
#         prompt = format_prompt()

#         # LLM initialization
#         llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.1, max_tokens=1500, timeout=30, streaming=True)

#         # Chain initialization
#         chain = prompt | llm

#         # Chain invoking
#         response = chain.invoke({"question" : question, "context" : context})

#         return {'Question' : question,
#                 'Web Search' : web_docs,
#                 'Ranked Docs' : rank_docs,
#                 'Prompt' : prompt,
#                 'Answer' : response.content
#                 }

In [ ]:
import time

def RagPipeline(question, top_n=2, sleep_time=15):
    if question:
        try:
            # Searching Web
            web_docs = search_web(question)
            time.sleep(sleep_time)  # Rate limit

            if not web_docs:
                return {'Question': question, 'Error': 'No results retrieved from the web.'}

            # Formating documents by extracting the snippets
            formatted_docs = format_document(web_docs)

            # Ranking the documents
            ranked_docs = rank_documents(question, formatted_docs)

            # Select Top-N documents and compile context
            context = compile_context(ranked_docs, top_n)

            # Create Prompt
            prompt = format_prompt()

            # LLM initialization
            llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.1, max_tokens=1500, timeout=30, streaming=True)

            # Chain initialization
            chain = prompt | llm

            # Chain invoking
            response = chain.invoke({"question": question, "context": context})

            return {
                'Question': question,
                'Web Search': web_docs,
                'Ranked Docs': [{'Document': doc, 'Score': score} for doc, score in ranked_docs],
                'Prompt': prompt.input_variables,
                'Answer': response.content
            }
        except Exception as e:
            return {'Question': question, 'Error': str(e)}



In [28]:
question = "Explain the impact of climate change on agriculture."
response = RagPipeline(question)

In [29]:
response

{'Question': 'Explain the impact of climate change on agriculture.',
 'Web Search': [{'snippet': 'The effects of climate change on agriculture will depend on the rate and severity of the change, as well as the degree to which farmers and ranchers can adapt. 7 U.S. agriculture already has many practices in place to adapt to a changing climate, including crop rotation and integrated pest management. A good deal of research is also under way ...',
   'title': 'Climate Change Impacts on Agriculture and Food Supply',
   'link': 'https://www.epa.gov/climateimpacts/climate-change-impacts-agriculture-and-food-supply'},
  {'snippet': 'There are numerous effects of climate change on agriculture, many of which are making it harder for agricultural activities to provide global food security.Rising temperatures and changing weather patterns often result in lower crop yields due to water scarcity caused by drought, heat waves and flooding. [5] These effects of climate change can also increase the ri

In [ ]:
# questions = [
#     "Why did China enact the one-child policy?",
#     "What are the benefits of renewable energy?",
#     "How does quantum computing work?",
#     "What caused the 2008 financial crisis?",
#     "Explain the impact of climate change on agriculture."
# ]

#### Rough Code

In [35]:
question = "Why did China enact the one-child policy?"
web_docs = search_web(question)
formatted_docs = format_document(web_docs)

rank_docs_ = rank_documents(question, formatted_docs)
context = "\n".join([doc for doc, score in rank_docs_[:2]])


In [36]:
rank_docs_

[("Origins of the One Child Policy. China's One Child Policy was one of the most controversial social experiments in modern history.Implemented to control population growth, it had far-reaching effects on Chinese society.. The policy was introduced in 1979 by the Chinese government to curb the rapid population growth.; It was part of a broader family planning policy that began in the early 1970s.",
  0.9532032608985901),
 ("one-child policy, official program initiated in the late 1970s and early '80s by the central government of China, the purpose of which was to limit the great majority of family units in the country to one child each. The rationale for implementing the policy was to reduce the growth rate of China's enormous population.It was announced in late 2015 that the program was to end in early 2016.",
  0.9510537385940552),
 ('A propaganda painting in Guangdong promotes the idea of a nuclear family with a single child. The text reads "Planned child birth is everyone\'s respon

In [38]:
context

"Origins of the One Child Policy. China's One Child Policy was one of the most controversial social experiments in modern history.Implemented to control population growth, it had far-reaching effects on Chinese society.. The policy was introduced in 1979 by the Chinese government to curb the rapid population growth.; It was part of a broader family planning policy that began in the early 1970s.\none-child policy, official program initiated in the late 1970s and early '80s by the central government of China, the purpose of which was to limit the great majority of family units in the country to one child each. The rationale for implementing the policy was to reduce the growth rate of China's enormous population.It was announced in late 2015 that the program was to end in early 2016."

C:\Users\Adeel Afzal\AppData\Local\Temp\ipykernel_8196\4054751686.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, timeout=30, streaming=True)


In [15]:
print (question)
print (context)

What are the benefits of renewable energy?
Key benefits of renewable energy for people and the planet. A ll energy sources have an impact on our environment, and r e newable energy is no exception.While each renewable energy source has its own sp e c ificities and trade-offs, the advantages over the devastating impacts of fossil fuels are undeniable: lower use of water and land, less air and water pollution, reduced wildlife and ...
Some of the benefits of renewable energy hit headlines, others are well known. But some benefits aren't even primary benefits at all, and are happy by-products of other pursuits. So for this week's Top 10, we run through some of the benefits that renewable energy brings businesses, individuals, governments and countries, to name a few. 10.


In [29]:


prompt = PromptTemplate.from_template("""

Instructions : You are an intelligent bot. Give Answer from the context.
Question : {question}
Context : {context}
""")

In [30]:
chain = prompt | llm


In [31]:
response = chain.invoke({
    "question" : question,
    "context" : context
})

In [33]:
context

"Key benefits of renewable energy for people and the planet. A ll energy sources have an impact on our environment, and r e newable energy is no exception.While each renewable energy source has its own sp e c ificities and trade-offs, the advantages over the devastating impacts of fossil fuels are undeniable: lower use of water and land, less air and water pollution, reduced wildlife and ...\nSome of the benefits of renewable energy hit headlines, others are well known. But some benefits aren't even primary benefits at all, and are happy by-products of other pursuits. So for this week's Top 10, we run through some of the benefits that renewable energy brings businesses, individuals, governments and countries, to name a few. 10."

In [32]:
response.content

'The benefits of renewable energy include:\n\n1. Lower use of water and land compared to fossil fuels.\n2. Reduced air and water pollution.\n3. Decreased negative impacts on wildlife.\n4. Positive economic effects for businesses and governments.\n5. Contribution to energy independence for countries.\n6. Creation of jobs in the renewable energy sector.\n7. Enhanced public health due to cleaner air and water.\n8. Long-term sustainability and reduced reliance on finite resources.\n9. Potential for technological innovation and advancements.\n10. Mitigation of climate change effects.'

In [80]:
context

"Renewable energy (or green energy) ... This has several benefits: electricity can move heat and vehicles efficiently and is clean at the point of consumption. [1] [2] Variable renewable energy sources are those that have a fluctuating nature, such as wind power and solar power.\nSome of the benefits of renewable energy hit headlines, others are well known. But some benefits aren't even primary benefits at all, and are happy by-products of other pursuits. So for this week's Top 10, we run through some of the benefits that renewable energy brings businesses, individuals, governments and countries, to name a few. 10."

In [30]:
questions = [
    "Why did China enact the one-child policy?",
    "What are the benefits of renewable energy?",
    "How does quantum computing work?",
    "What caused the 2008 financial crisis?",
    "Explain the impact of climate change on agriculture."
]